In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trainmain=pd.read_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\train.csv")
testmain=pd.read_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\test.csv")

In [3]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
    
def uniquevalues(df):
    for col in df.columns:
        if df[col].dtype=='O':
            print("The unique values of {0} is {1}".format(col,df[col].nunique()))
            
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category,drop_first=True)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns    

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

def dateformatchange(date):
    Month=date.split('-')[0]
    Year="20"+ date.split('-')[1]
    monthnum=d.get(Month)
    newdate= "0" + str(monthnum) + "/" + Year
    return newdate

def checkmvalue(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=0
    for i in allm:
        if df[i]>0:
            j=1
    return j  
def checknummissed(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=0
    for i in allm:
        if df[i]>0:
            j=j+1
    return j 

def checkmaxmissed(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=df[allm].max()
    return j

In [4]:
df=trainmain.copy()
df_test=testmain.copy()
df.drop('loan_id',axis=1,inplace=True)
df_test.drop('loan_id',axis=1,inplace=True)
df_test['origination_date'] =pd.to_datetime(df_test['origination_date']).dt.strftime('%Y-%d-%m')
d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12 }
df_test['first_payment_date']=df_test['first_payment_date'].apply(dateformatchange)

In [5]:
import datetime

now = datetime.datetime.now()
today=now.strftime("%Y-%m-%d")
df['loan_age_months'] =  (pd.to_datetime(today)-pd.to_datetime(df.origination_date))/np.timedelta64(1,'M')
df['loan_age_months'] = df['loan_age_months'].astype(int)
df['remaining_months']=df['loan_term']-df['loan_age_months']

In [6]:
df_test['loan_age_months'] =  (pd.to_datetime(today)-pd.to_datetime(df_test.origination_date))/np.timedelta64(1,'M')
df_test['loan_age_months'] = df_test['loan_age_months'].astype(int)
df_test['remaining_months']=df_test['loan_term']-df_test['loan_age_months']

In [7]:
df['mothly_amount']=(df['unpaid_principal_bal']/df['remaining_months'])

In [8]:
df_test['mothly_amount']=(df_test['unpaid_principal_bal']/df_test['remaining_months'])

In [9]:
#df['flagm']= df.apply(lambda row: checkmvalue(row), axis = 1)
#df_test['flagm']= df_test.apply(lambda row: checkmvalue(row), axis = 1) 
#df['flagmmissedno']= df.apply(lambda row: checknummissed(row), axis = 1)
#df_test['flagmmissedno']= df_test.apply(lambda row: checknummissed(row), axis = 1)
#df['flagmmaxmissed']= df.apply(lambda row: checkmaxmissed(row), axis = 1)
#df_test['flagmmaxmissed']= df_test.apply(lambda row: checkmaxmissed(row), axis = 1)

In [10]:
X=df.drop('m13',axis=1)
y=df['m13']


In [11]:
X, cat_cols = one_hot_encoder(X, False)
df, cat_cols = one_hot_encoder(df, False)
df_test, cat_cols = one_hot_encoder(df_test, False)

In [12]:
X=X.drop(['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','insurance_type'],axis=1)
df_test=df_test.drop(['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','insurance_type'],axis=1)

In [13]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)

reduc_estimator =LGBMClassifier(
    max_depth= -1,
    boosting_type= 'gbdt',
    objective= 'binary', 
    nthread= 5, 
    num_leaves= 27,
    learning_rate= 0.0085, 
    max_bin= 512, 
    subsample_for_bin= 200,
    subsample= 0.7, 
    subsample_freq= 1, 
    colsample_bytree= 0.65, 
    reg_alpha= 1.2, 
    reg_lambda= 1.2, 
    min_split_gain= 0.5, 
    min_child_weight= 1,
    min_child_samples= 5, 
    scale_pos_weight= 1,
    num_class= 1, 
    metric= 'binary_error',
    n_estimators= 100)

# Parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [x for x in range(36, 46, 2)],
    'learning_rate': [0.0085,0.085,0.01,0.10]
    
}

gridsearch = GridSearchCV(reduc_estimator, param_grid,cv=5)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_test, y_test)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 2000)
print('Best parameters found by grid search are:', gridsearch.best_params_)

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[1]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.875115	valid_0's binary_logloss: 0.031813
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.878252	valid_0's binary_logloss: 0.031695
[3]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.883695	valid_0's binary_logloss: 0.0306307
[4]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.892268	valid_0's binary_logloss: 0.029781
[5]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.893467	valid_0's binary_logloss: 0.0296889
[6]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.90124	valid_0's binary_logloss: 0.0296175
[7]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.902004	valid_0's binary_logloss: 0.0289944
[8]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.902938	valid_0's binary_logloss: 0.0284583
[9]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.903002	valid_0's binary_logloss: 0.0280104
[10]	valid_0's binary_error: 0.00534206	valid_0's

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[1]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.848155	valid_0's binary_logloss: 0.0318543
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.855799	valid_0's binary_logloss: 0.0307514
[3]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.863178	valid_0's binary_logloss: 0.0299188
[4]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.879419	valid_0's binary_logloss: 0.0298095
[5]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.88558	valid_0's binary_logloss: 0.0297088
[6]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.883392	valid_0's binary_logloss: 0.0290645
[7]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.888733	valid_0's binary_logloss: 0.0285525
[8]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.892011	valid_0's binary_logloss: 0.0281
[9]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.891242	valid_0's binary_logloss: 0.0280614
[10]	valid_0's binary_error: 0.00534206	valid_0's

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

[1]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.789823	valid_0's binary_logloss: 0.0320153
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.87054	valid_0's binary_logloss: 0.0221063
[3]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.871745	valid_0's binary_logloss: 0.0216617
[4]	valid_0's binary_error: 0.00534206	valid_0's auc: 0.882344	valid_0's binary_logloss: 0.0214514
[5]	valid_0's binary_error: 0.00516974	valid_0's auc: 0.878732	valid_0's binary_logloss: 0.021299
[6]	valid_0's binary_error: 0.00508358	valid_0's auc: 0.877137	valid_0's binary_logloss: 0.0209831
[7]	valid_0's binary_error: 0.00499742	valid_0's auc: 0.875864	valid_0's binary_logloss: 0.0207667
[8]	valid_0's binary_error: 0.00473893	valid_0's auc: 0.876774	valid_0's binary_logloss: 0.020567
[9]	valid_0's binary_error: 0.00465277	valid_0's auc: 0.874939	valid_0's binary_logloss: 0.0204796
[10]	valid_0's binary_error: 0.0045666	valid_0's

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[1]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.858059	valid_0's binary_logloss: 0.0212116
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.858763	valid_0's binary_logloss: 0.0209834
[3]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.880846	valid_0's binary_logloss: 0.0207996
[4]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.884825	valid_0's binary_logloss: 0.0206044
[5]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.880237	valid_0's binary_logloss: 0.0204546
[6]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.878665	valid_0's binary_logloss: 0.0203318
[7]	valid_0's binary_error: 0.00439428	valid_0's auc: 0.876698	valid_0's binary_logloss: 0.0202222
[8]	valid_0's binary_error: 0.00448044	valid_0's auc: 0.881001	valid_0's binary_logloss: 0.0201033
[9]	valid_0's binary_error: 0.00439428	valid_0's auc: 0.88018	valid_0's binary_logloss: 0.0200425
[10]	valid_0's binary_error: 0.00430812	valid_0's au

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[1]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.881171	valid_0's binary_logloss: 0.0217555
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.898056	valid_0's binary_logloss: 0.0213547
[3]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.894399	valid_0's binary_logloss: 0.0211161
[4]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.893494	valid_0's binary_logloss: 0.0209131
[5]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.889736	valid_0's binary_logloss: 0.0206825
[6]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.887287	valid_0's binary_logloss: 0.0204837
[7]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.88123	valid_0's binary_logloss: 0.0203424
[8]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.882827	valid_0's binary_logloss: 0.0202497
[9]	valid_0's binary_error: 0.0045666	valid_0's auc: 0.883377	valid_0's binary_logloss: 0.0201462
[10]	valid_0's binary_error: 0.0045666	valid_0's auc: 0

In [14]:
pred=gridsearch.best_estimator_.predict(df_test)

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
gridsearch.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
        importance_type='split', learning_rate=0.1, max_bin=512,
        max_depth=-1, metric='binary_error', min_child_samples=5,
        min_child_weight=1, min_split_gain=0.5, n_estimators=36, n_jobs=-1,
        nthread=5, num_class=1, num_leaves=27, objective='binary',
        random_state=None, reg_alpha=1.2, reg_lambda=1.2,
        scale_pos_weight=1, silent=True, subsample=0.7,
        subsample_for_bin=200, subsample_freq=1)

In [16]:
pred[pred==1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int64)

In [19]:
sub=pd.DataFrame(columns=['loan_id','m13'])
sub['loan_id']=testmain['loan_id']
sub['m13']=pred
sub.to_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\sub.csv",index=False)

In [318]:
X['debt_to_income_ratio'].max()

64.0